# Word Embedding Tutorial

## What you can expect from this tutorial
At the end of this notebook, you'll have an understanding of
- what is a word vector, and why do I need one?
- how to get word vectors
    - distributional semantics: simple counts
    - word embeddings: neural methods
- how to use word embeddings
    - finding similar terms
    - combining concepts and searching for terms between them, "more like me"
    - using negative examples to complete analogy questions (sometimes)

## What's out-of-scope for this tutorial?
You won't learn about
- different methods for training neural word embeddings
    - CBOW/Skip-gram
    - Word2Vec
    - How FastText works
- Using word embeddings for downstream classification tasks
- Measuring the goodness of your embedding model
    - detecting unethical bias in your model
    - comparing one word embedding model to another to determine which is better

In [1]:
# let's start a dictionary so we can store definitions for words we don't know
d = {}

## Why do we need word vectors?
Remember feature vectors? We have this list of numbers that represents an instance of data. This numerical representation allows us to make predictions using predictive modeling algorithms. 

Consider the feature vectors for cars.

|   instances   | is_red      | has_4_wheel_drive | label |
| ------------- | ----------- | ----------- | ---------|
| instance_1    | 0           | 1 | truck |
| instance_2    | 1           | 0 | car |

A word vector is a way to represent the features of this word and (maybe) use in a downstream modeling task. Word vectors allow us to represent the meaning of a word numerically. With this vector represation of a word's semantics, we can easily give this information to predictive models.

A word vector might look like this

|   words   | appeared_near_yum      | appeared_near_red |
| ------------- | ----------- | ----------- |
| apple    | 1           | 1 |
| applesauce    | 1           | 0 |
| firetruck | 0 | 1 |

This is in accordance with the hypothesis of [distributional semantics](https://en.wikipedia.org/wiki/Distributional_semantics#:~:targetText=Distributional%20semantics%20is%20a%20research,large%20samples%20of%20language%20data.): _linguistic items with similar distributions have similar meanings_

A _word vector_ uses the distributional hypothesis. Each word vector represents a word with a very high-dimensional sparse vector, where each dimension reflects a context in which the word occurred in the corpus. For example, a context could be another word that appeared in proximity.

A _word embedding_ represents a word with a low-dimensional vector (e.g. 100 dimensions). The dimensions are usually latent, and often obtained using the information as in the distributional semantics approach (e.g. LSA, word2vec). ([source](https://www.quora.com/Whats-the-difference-between-word-vectors-word-representations-and-vector-embeddings))

Word embeddings are often used as a feature extraction technique. But the embeddings themselves are useful too!

In [2]:
# update the definition dictionary with new terms 
# distributional hypothesis, distrubtional semantics, word vector, word embedding
d["distributional hypothesis"] = "linguistic items with similar distributions have similar meanings"
d["distributional semantics"] = "linguistic theory that relies on dist. hypothesis"
d["word vector"] = "high dimensianal sparse vector that represents a word. Dimensions have real-world  meaning  and are equal to  the vocab length"
d["word embedding"] = "a low dimensional vector that represents a word. Dimensions are abstract"

In [3]:
# you may want these imports for the next coding section
from collections import defaultdict
import numpy as np
import os

In [17]:
# implement this simple way to make a word vector using a corpus
'''
----------------------------------------------
Objective: make word vectors using context
----------------------------------------------
create a function that takes a file of sentences and returns
a dictonary mapping each word in the corpus to its vector
```
>>> vectors = make_simple_word_vectors(filename)
>>> vectors['dog']
np.array([0., 0., 0., 0., 0., 0., 2., 0., 0., 0., 
         0., 0., 0., 1., 1., 0., 4., 0., 0., 0.])
```
To make sure everyone has the same columns, use a helper function,
`make_vocab(filename)` which returns a map of the vocab item to the index
of the column it corresponds to. Sort the vocab so that the index of 'a' is 0
```
>>> vocab = make_vocab(filename)
>>> vocab['a']
0
>>> vocab['dog']
9
```
You're welcome to use other helper methods.

For your corpus, use the file: data/10_sentences.txt
'''

def make_vocab(filename):
    """
    :param filename: the location fo the file with the word data to build vectors from
    :return: dictionary mapping each vocab item to it's index
    """
    vocab = set()
    with open(filename, 'r') as f:
        for line in  f:
            words = line.lower().split()
            vocab.update(words)
            
    sorted_vocab = sorted(vocab)
    
    dictionary = {}
    for i, word in enumerate(sorted_vocab):
        dictionary[word] =  i
    return dictionary

def make_simple_word_vectors(filename, window_size=2):
    """
    :param filename: the location of the file with word data to build vectors from
                     the file should be formatted with each sentence on a line
    :param window_size: how far to the left and right of a word to look for context
                        for window_size=2, we look 2 words to the left, and 2 words right
    :return: dictionary mapping a word to its vector
             the columns in the vector should be ordered alphabetically
    """
    vocab = make_vocab(filename)
    num_words  = len(vocab)
    
    vectors =  defaultdict(lambda: np.zeros(num_words))
    with open(filename, 'r') as f:
        for line in f:
            words =  line.lower().split()
            for i, word in enumerate(words):
                before_index = max(i - window_size, 0)
                after_index = min(i + window_size, len(words))

                for x in range(before_index, after_index):
                    if x == i: continue
                    index = vocab[words[x]]
                    vectors[word][index] += 1
                    
    return vectors
                    

# always debug with a small file first
small_file = os.path.join('data', '10_sentences.txt')
vocab = make_vocab(small_file)
assert(len(vocab) == 20)

word_to_vector_map = make_simple_word_vectors(small_file)
assert(np.array_equal(word_to_vector_map['dog'],
                      np.array([0., 0., 0., 0., 0., 0., 2., 0., 0., 0., 
                                0., 0., 0., 1., 1., 0., 4., 0., 0., 0.])))

## We've incorporated a word's context into the feature vector
> _"You shall know a word by the company it keeps." - John Rupert Firth_

We need the word vector to capture the similarity between words. From linguistics, a simple approach is to say that words that fall in similar contexts have similar meanings. The word vectors we've created capture this notion of contextually similar terms. 

## Pros and Cons

Pros
 - captures the context of words
 
Cons
 - [homographs](https://en.wikipedia.org/wiki/Homograph) have the same vector
 - [sparse vectors](https://en.wikipedia.org/wiki/Sparse_matrix) are expensive
     - could do a dimensionality reduction technique like [SVD](https://en.wikipedia.org/wiki/Singular_value_decomposition)
     
In practice, the sparsity of the vectors on large corpora make using them impossible. 
(That's why we only used 10 sentences.)

In [ ]:
# update dictionary of new terms
# homograph, sparse vector
# don't worry about SVD - not important for what we're doing
d["homograph"] = "words that are spelled the same but have different meaning"
d["sparse vector"] = "too many 0s"

## Neural Word Embeddings
In practice, the count method can be improved upon, both computatationally and theoretically.

Instead of counting up words near other words, let's use a neural network to get word embeddings.

_But... don't we need labeled data?_

This is a form of [semi-supervised learning](https://en.wikipedia.org/wiki/Semi-supervised_learning#:~:targetText=Semi%2Dsupervised%20learning%20is%20a,large%20amount%20of%20unlabeled%20data.), which makes use of unlabeled data for training.

In [ ]:
# update dictionary of new terms
# semi-supervised learning
d["semi-supervised learning"] = "collection of ML algorithms that use data the 'labels' itself"

_So... what are we predicting then?_

Let's hide one of the words in a data instance.

> I need to feed my ____ today

Humans have no problem knowing what kinds of words might fill this blank, and knowing words that do _not_ fit in this blank. Let's train a neural network to predict this missing word given the context. This method is called CBOW, and we'll go into more depth in day 2. 

![CBOW](figures/word2vec-cbow.png)

_Okay, great, we have this model that fills in the blanks. How does that get us word vectors?_

We use the Weight matrix W. Each row in this weight matrix corresponds to a word in the vocab from the corpus a model was trained on.

![weight_matrix](figures/weight-matrix.png)

In [18]:
# say we have a weight matrix W and a one-hot encoding for "hello"
W = np.array([[.47, .56, .98, -.23],
              [-.21, -.79, -.99, 1.2],
              [-3.2, -.09, .09, .19],
              [-3.2, -.89, .89, .19],
             ])
hello = np.array([0, 0, 0, 1])
# QUIZ: how many words are in the vocab?
# to get the word embedding for hello, we need to multiply
hello_embedding = np.matmul(hello, W)
print(f"hello_embedding = {hello_embedding}")

hello_embedding = [-3.2  -0.89  0.89  0.19]


Let's implement a neural word embedding model using [FastText](https://arxiv.org/pdf/1607.04606.pdf). 

But first, we need to implement something to preprocess the data more efficiently. Holding all the data in memory at once may lead to a memory error. Instead we want to stream the data, only looking at one line in the file at a time.

### Iterators and Generators in Python
Python cares more about how objects behave rather than what they are. 

For instance, you can iterate through a list, and you can iterate through a dictionary's keys and values. These objects are _iterable_. They have a class method `__iter__()` and can be called like `for x in iterable:`. 

![Generators and Iterators](figures/iterators.png)

[Reference](https://nvie.com/posts/iterators-vs-generators/)

Generators are iterators, but instead of holding every line in memory, they compute it on the fly. They still have an `__iter__()` class method and word in `for x in generator:` calls, but they are much less memory intensive. 

Generators make use of the `yield` keyword, which then returns to the outer call of iteration with only that line. When the iterable needs to call the next item, it returns to the line immediately after the yield and continues. 

If a `return` statement is present in a generator function, then a `StopIteration` is raised, and the iteration is halted. 

In [ ]:
# generator examples

def my_generator():
    yield 1
    yield 2
    
for x in my_generator():
    print(f"x={x}")

In [ ]:
nums = my_generator()

for x in nums:
    print(f"x={x}")
    
for x in nums:
    print(f"x={x}")
    
print("done")

#### Quiz: Why didn't it go through twice?

For this reason, we'll create a class that has an `__iter__()` method that we'll override. This way, every time we call `for sent in sentences:`, we'll have data.

In [22]:
# See Day 2's tutorial for more algorithms to train word embeddings
# for more on different training algorithms

'''
-------------------------------------------------------
Objective: create a generator for better performance
-------------------------------------------------------
first, we need to preprocess the data. 

The input to FastText is an iterable of sentences 
where a sentence is formatted as: 
`['this', 'is', 'a', 'sample', 'sentence']`

So you could use a list of sentences 
```
sentences = [
    ['this', 'is', 'a', 'sample', 'sentence'],
    ['this', 'is', 'another', 'sample', 'sentence'],
]
```
but holding all of the corpus's sentences in memory at once is expensive.
You might hit a memory error.

Instead, let's use a generator that yields data on the fly.

Create a class `Sentences` that can be iterated through to train a model.
Make sure your object yields preprocessed data. 
That might mean all lowercase, no punctuation, but you can make your own choices
with what you think will work best.

```
>>> corpus = os.path.join('data', 'got_scripts_breakdown.csv')
>>> sentences = Sentences(filename=corpus,
                          delim=';',
                          encoding='utf-8-sig',
                          limit=5,
                          )
>>> for sent in sentences:
        print(sent)
['what', 'd’you', 'expect', 'they’re', 'savages', 'one', 'lot', 'steals', 'a', 'goat', 
 'from', 'another', 'lot', 'and', 'before', 'you', 'know', 'it', 'they’re', 'ripping', 
 'each', 'other', 'to', 'pieces']
['i’ve', 'never', 'seen', 'wildlings', 'do', 'a', 'thing', 'like', 'this', 'i’ve', 
 'never', 'seen', 'a', 'thing', 'like', 'this', 'not', 'ever', 'in', 'my', 'life']
['how', 'close', 'did', 'you', 'get']
['close', 'as', 'any', 'man', 'would']
['we', 'should', 'head', 'back', 'to', 'the', 'wall']
```
'''

import string


class Sentences:
    def __init__(self, filename, delim, encoding, limit=float('inf')):
        self.filename = filename
        self.delim = delim
        self.encoding = encoding
        self.limit = limit
        
    def preprocess(self, line, delim, sent_index):
        line = line.lower()
        columns = line.split(self.delim)
        sentence = columns[sent_index]
        words = sentence.split()
        return words
        
    def __iter__(self):
        with open(self.filename, 'r') as f:
            # header we need to skip
            line = next(f)
            cols = line.split(self.delim)
            sent_index = cols.index("Sentence")
            # need to extract the right column for snetence
            
            for i, line in enumerate(f):
                if i > self.limit:
                    return
                yield self.preprocess(line, self.delim, i)
        
                

got_dialogue_file = os.path.join('data', 'got_scripts_breakdown.csv')
sents = Sentences(filename=got_dialogue_file, 
                  delim=';', 
                  encoding='utf-8-sig',
                  limit=5)
for sent in sents:
    print(sent)

['0']
['season', '1']
['winter', 'is', 'coming']
['do', 'the', 'dead', 'frighten', 'you?']
['gared']
['1']


Great, our data is preprocessed, and now we're ready to train a model.
Check out [gensim's documentation](https://radimrehurek.com/gensim/models/fasttext.html) too for more information on the hyperparameters you can tune while training. 

In [26]:
'''
-------------------------------------------------
objective: train a FastText word embedding model
-------------------------------------------------
use gensim to train a model. Follow the documentation for an example
'''

from gensim.models import FastText
sentences = Sentences(got_dialogue_file, delim=";", encoding="utf-8-sig")
model = FastText(size=100, window=8, min_count=4)
model.build_vocab(sentences)
model.train(sentences=sentences, total_examples=model.corpus_count, epochs=100)


IndexError: list index out of range

## What's happening when we train?

We then iterate through the data, to build the vocabulary. This determines which words are frequent enough to build vectors for, which words will be included in the model. 

At the start of training the model, the weights for the word embeddings are randomly initialized. Then the model iterates through the data, making predictions. 

![Prediction/Inference Step](figures/nn-inference.gif)

Then we evaluate how far off we were using the cost function.
![Calculating the loss](figures/nn-cost.gif)

After training completes, we take the weight matrix. Each row corresponds to a word in our vocabulary. These vectors live in 300 dimensional space!

In [ ]:
# you can accesss the vectors from the model using model.wv
# and keying in like a dictionary


In [ ]:
# what about words that aren't in the corpus?


## What can we do with word embeddings?

Similar vectors imply semantically similar concepts. Vectors that live in the same space mean very similar things. Let's use the similarity of vectors to find similar words.

What makes a vector similar to another one?

Let me show you. Our vectors had a large dimension (100? 300?), but that's difficult to visualize. We can use an algorithm like [t-SNE](https://en.wikipedia.org/wiki/T-distributed_stochastic_neighbor_embedding) to visualize in 2 or 3 dimensions more easily. 

Here's an example of word embeddings that have undergone t-SNE.
![word embeddings plotted](figures/word-embedding-clusters.png)

Words with similar meanings are close together in space! We lose some information in the dimension reduction to plot in 2d space -- which may be why the months are shown in two different places, but there may be another more meaningful reason.

_Okay, but how can I tell which things are close without that loss of information from the dimension reduction? I don't need a picture, I just want numbers._

We need to use a distance [metric](https://en.wikipedia.org/wiki/Metric_(mathematics)#:~:targetText=In%20mathematics%2C%20a%20metric%20or,of%20elements%20of%20a%20set.&targetText=A%20metric%20induces%20a%20topology,a%20metric%20is%20called%20metrizable.). The closer two vectors are, the more similar their meanings tend to be. 

Here's a dirty secret topologist's won't tell you: You can define distance however you want to. There's no universal distance metric that works well in all spaces, on all data sets, or on all concepts.

Let's talk about two commonly used distance metrics.

![vector similarity](figures/The-difference-between-Euclidean-distance-and-cosine-similarity.png)

| Metric        | pro           | con   |
| ------------- |-------------| -----|
| euclidean     | distance ranges from \[0, inf\]             | takes the magnitude of the vectors into account |
| cosine        | captures difference in direction of vectors | bounded \[0,2\] |

In practice, cosine similarity _usually_ gives more insightful results than euclidean, likely because of the issues with vectors of significantly different magnitudes.

In [ ]:
# update new terms dictionary 
# distance metric
d["distance metric"] = "how to measure the space between two vectors"

In [25]:
# gensim builds in this funcitonality for you! Checkout model.wv.most_similar()
# they use cosine similarity, a score between 0 and 1, 1 meaning very similar

# CAUTION: there's also cosine _distance_ which returns a score between 0 and 2 
#          where 0 means most similar. Always double check your cosine function
# we've checked for you: gensim uses cosine similarity
from gensim.models import FastText
model.wv.most_similar("dragon")

NameError: name 'model' is not defined

In [ ]:
# we can even plot these vectors in space using t-SNE
# we've implemented this funciton for you for the sake of time

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import os
from gensim.models import FastText
import pandas as pd
from sklearn.manifold import TSNE

def plot_word_vectors(word_lists, model):
    all_words = [word for words in word_lists for word in words]
    
    X = [model.wv[word] for word in all_words]
    tsne = TSNE(n_components=2)
    X_tsne = tsne.fit_transform(X)
    
    df = pd.DataFrame(X_tsne, index=all_words, columns=['x', 'y'])
    
    dfs = [df.loc[words] for words in word_lists]
    
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    
    colors = cm.rainbow(np.linspace(0,1, len(dfs)))
    for df, color in zip(dfs, colors):
        ax.scatter(df['x'], df['y'], color=color)
    
    for df in dfs:
        for word, row in df.iterrows():
            plt.text(row['x'], row['y'], word)
    plt.show()

words = [
    [word for word, _ in model.wv.most_similar('winter', topn=20)],
    [word for word, _ in model.wv.most_similar('dragon', topn=20)],
    [word for word, _ in model.wv.most_similar('wedding', topn=20)],
]
plot_word_vectors(words, model)

#### What is lost with a dimensionality reduction technique?

Think about looking at this data set from different angles. 
![3d data](figures/3d-scatter-plot-rotate.gif)

When you're viewing high dimensional data in lower dimensions, you don't get the best idea for what's really happening. But it can be insightful.

### Searching for more than one word at a time

In [ ]:
# gensim also allows you to search for multiple concepts


How does it know what to pull back for more than one word?

It takes an average of the vectors provided in the list of positive objects. 

In [ ]:
# let's show this in 2 dimensions
v1 = np.random.uniform(low=-5, high=5, size=2)
v2 = np.random.uniform(low=-5, high=5, size=2)
print(f"v1: {v1}")
print(f"v2: {v2}")

avg_v = np.average([v1, v2], axis=0)
print(f"avg_v: {avg_v}")

V = [v1, v2, avg_v]

# let's plot these vectors using matplotlib
plt.quiver(v1[0], v1[1], color='r', scale=21)
plt.quiver(v2[0], v2[0], color='b', scale=21)
plt.quiver(avg_v[0], avg_v[1], color='g', scale=21)
plt.show()

Does the average vector always look like it's it in the right spot? 

There may be a better function to use, but gensim implemented it this way. 

When two vectors are close together, this functionality makes a lot of sense. Find the word most similar to the average of these two words. But when two vectors are pointing in opposite directions, the average would be the origin, something kind of far away from both of those vectors. When giving examples, make sure your examples are similar enough to get reasonable results. 

Gensim also offers the ability to include negative examples

In fact, there are many papers publishing results from training word embeddings with such fantastic examples as `"king" - "man" + "woman" ≈ "queen"`. Let's see how this work on our data set. 

In [ ]:
# implement the analogy by using 'king' and 'woman' in the positive params
# and 'man' in the negative param


This result is amazing (and famous).

![analogies](figures/king-queen-analogy.png)

But does it always work?

In [29]:
# try a couple other anologies to see if they work!
# here's one to get you started 

model.wv.most_similar(positive=['fight'], negative=['smart'])

[]

In [ ]:
# can you find an analogy that works?

Not as much luck in other analogy examples.

This [paper](https://kawine.github.io/blog/nlp/2019/06/21/word-analogies.html) proves the conditions necessary for these incredible analogies to work. In short, more data likely solves this problem. 

![When analogies work](figures/when-analogies-work.png)


In research, we've tried to figure out the best way to incorporate negative examples into our searches, but we just haven't found a consistently useful result. We also aren't using our word embeddings for analogies, so we don't want to get too hung up on this. 

In [32]:
# let's save our model for further use
# create the directory 'models'
# and save the model as 'got_ft.model'

save_path = os.path.join('models', 'got_ft.model')
if not os.path.isdir("modles"):
    os.mkdir("models")
model.save(save_path)


FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'models'

### Other values of word embeddings coming from automatically transcribed speech data

- we capture aliases! 
- we get tons of insight from the specific context of call centers

# Summary - What did we learn today?
- ways of training word embeddings
    - raw counts from the data
        - can't handle out-of-vocab words
        - captures contextually similar words
    - neural methods: FastText
        - captures character similarities as well as contextual similarities
- how to use word embeddings
    - find similar words
    - search for more than one word at a time
    - search for "a", and "b", but not like "c" with questionable results

In [ ]:
# let's review our dictionary of new terms
from pprint import pprint


# Survey !!!

Please complete the [course survey](https://forms.office.com/Pages/ResponsePage.aspx?id=gwv7BWBlfUGFbTjusOst_QYpnoW2nrtJmgVZLQ3gu25UMURGMDdaUTA0QUhJQTM3NlMxNE9GVVkyRC4u)